### Feature selection--wrapper model
* This notebook iteratively select 50 most significant features out of the pre-selected 498 numerical features using inertia score of the K-means algorithm as criteria. The algorithm was run on a subsample of the original data (1.25% of the original data set).

In [7]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist, pdist
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.cm as cm

In [8]:
# Use a smaller data set to save time
df = pd.read_csv('PHBsample14_sss.csv', low_memory=False)
# drop the column resulted from sampling of the original data set
df.drop('Unnamed: 0', axis=1, inplace=True)
# In order to run K-means, drop all the categoricald data for now.
df = df.select_dtypes(include=['float64', 'int64'])
# Impute missing values with means
df = df.fillna(df.mean())

In [9]:
selected_variable = pd.read_csv('selectedVariables.csv')
selected_variable.drop('Unnamed: 0', axis=1, inplace=True)
df1 = df[df.columns.intersection(selected_variable.columns)]
PolNum = np.asarray(df1.PolNum_UW)
df2 = df1.iloc[:, :-1]
df3 = StandardScaler().fit_transform(df2)

In [10]:
X = pd.DataFrame(df3, columns = df2.columns)

In [11]:
n, m = X.shape[0], X.shape[1]
print(n, m)

59159 498


In [5]:
model_test = KMeans(n_clusters=7)
model_test.fit(X) 
pred_y=model_test.labels_

In [9]:
print("Clustered class labels:", "\n", pd.value_counts(pd.Series(pred_y)))

Clustered class labels: 
 0    15491
5    14160
4     8910
6     5925
3     5687
1     5313
2     3673
dtype: int64


In [10]:
X.columns

Index(['ValDate', 'IssDate', 'IssAgeALB', 'Dur', 'AttAge', 'JointInd', 'AV',
       'CSV', 'SCPeriod', 'WDtoDate',
       ...
       'Match4', 'tie3', 'HealthScore_C5', 'Surr', 'EligibleInd', 'WDResponse',
       'FirstEligQInd', 'UtilizationInd', 'WDModelFilterIn', 'PolNum_UW'],
      dtype='object', length=907)

In [15]:
X.head()

,ValDate,IssDate,IssAgeALB,Dur,AttAge,JointInd,AV,CSV,SCPeriod,WDtoDate,...,Match4,tie3,HealthScore_C5,Surr,EligibleInd,WDResponse,FirstEligQInd,UtilizationInd,WDModelFilterIn,PolNum_UW
0,16343.0,16104.0,65.0,1.0,65.8,0.0,448559.96,421076.98,5,0.00,...,1.0,64.859049,0.500000,0.0,1.0,0.0,0.0,0.0,1.0,294692
1,15613.0,14397.0,69.0,4.0,72.4,0.0,67321.31,64451.77,7,0.00,...,1.0,53.000000,0.869053,0.0,1.0,0.0,0.0,0.0,1.0,281394
2,16070.0,13518.0,55.0,7.0,62.6,0.0,301121.04,295758.92,7,56438.97,...,1.0,57.000000,0.500000,0.0,1.0,0.0,0.0,1.0,0.0,475776
3,16343.0,14419.0,53.0,6.0,58.3,0.0,187344.04,180762.56,7,0.00,...,1.0,104.000000,0.500000,0.0,1.0,0.0,0.0,0.0,1.0,288738
4,15613.0,15044.0,63.0,2.0,65.2,0.0,183155.51,171789.66,6,1845.38,...,0.0,64.859049,0.869053,0.0,1.0,0.0,0.0,1.0,1.0,15320


In [13]:
# let's assume there are 7 clusters
num_of_cluster = 7
# Let's assume we're going to select 50 features out of 498 features, therefore we're going to iterate 50 times
num_of_iter = 50
model = KMeans(n_clusters=num_of_cluster)
score = np.zeros([num_of_iter, m]) # the sum of squared distances of samples to their closest cluster center
exclude_columns = [] # best performed models with selected features will be added to this list after every iteration
include_columns = [i for i in range(np.shape(score)[1]) if i not in exclude_columns] # rest of the features

for iteration in range(num_of_iter):
    # The first iteration, we're going to test clustering models on each individual variables
    if iteration == 0:
        print("Now processing iteration %d" %iteration, "\n")   
        for i in range(m):
            model.fit(X.iloc[:, i][:, np.newaxis])
#             pred_y = model.labels_
#             print("cluster labels based on variable %s:" %X.columns[i], "\n", pd.value_counts(pd.Series(pred_y)))
            score[iteration][i] = model.inertia_
#             print("the sum of squared distances of samples to their closest cluster center based on variable %s" \
#                  %X.columns[i], "is:", score[iteration][i])   
        
        selected_feature_index = np.argmin(score[iteration], axis=0) 
        selected_feature_score = np.amin(score[iteration], axis=0) 
        selected_feature = X[X.columns[selected_feature_index]]
        exclude_columns.append(selected_feature_index)
        print("Conclusion: cluster based on variable %s" %X.columns[selected_feature_index], "gives the best performance", "\n") 
    #for following iteration, we're going to add the rest the feature to the selected feature and perform cluster model
    else:
        print("Now processing iteration %d" %iteration, "\n") 
        for i in range(m):
            if i not in exclude_columns:
                # Generate data with features selected from last iteration plus each individual rest of the features
                data = pd.concat([selected_feature, X[X.columns[i]]], axis=1)
                model.fit(data)
#                 pred_y = model.labels_
#                 print("cluster labels based on variables:", data.columns, "\n", pd.value_counts(pd.Series(pred_y)))
                score[iteration][i] = model.inertia_
#                 print("the sum of squared distances of samples to their closest cluster center based on variables:", \
#                  data.columns, "is:", score[iteration][i]) 
        include_columns = [i for i in range(np.shape(score)[1]) if i not in exclude_columns]
        selected_feature_score = np.amin(score[:,include_columns][iteration], axis=0) 
        selected_feature_index = np.argmin(score[:,include_columns][iteration], axis=0) 
        selected_feature = pd.concat([selected_feature, X[X.columns[selected_feature_index]]], axis=1)
        exclude_columns.append(selected_feature_index)
        print("Conclusion: cluster based on variable %s" %X.columns[exclude_columns], "gives the best performance", "\n") 
print("Selected features are %s" %X.columns[exclude_columns])

Now processing iteration 0 

Conclusion: cluster based on variable JointInd gives the best performance 

Now processing iteration 1 

Conclusion: cluster based on variable Index(['JointInd', 'MarkettoArea_C1'], dtype='object') gives the best performance 

Now processing iteration 2 

Conclusion: cluster based on variable Index(['JointInd', 'MarkettoArea_C1',
       'Housing.Units.Percent.Built.2000.to.2004_C3'],
      dtype='object') gives the best performance 

Now processing iteration 3 

Conclusion: cluster based on variable Index(['JointInd', 'MarkettoArea_C1',
       'Housing.Units.Percent.Built.2000.to.2004_C3',
       'Housing.Units.Percent.5.Units_C3'],
      dtype='object') gives the best performance 

Now processing iteration 4 

Conclusion: cluster based on variable Index(['JointInd', 'MarkettoArea_C1',
       'Housing.Units.Percent.Built.2000.to.2004_C3',
       'Housing.Units.Percent.5.Units_C3', 'Number.of.Lifestyles.All_C3'],
      dtype='object') gives the best performa

Conclusion: cluster based on variable Index(['JointInd', 'MarkettoArea_C1',
       'Housing.Units.Percent.Built.2000.to.2004_C3',
       'Housing.Units.Percent.5.Units_C3', 'Number.of.Lifestyles.All_C3',
       'CEN_tr_pctGE200KAge65plus', 'CEN_tr_pctLT50KAge65plus',
       'CEN_tr_pctLT25KAge65plus', 'CEN_tr_pctLT10KAge65plus',
       'CEN_tr_pctProductionFamily', 'CEN_tr_pctConstructionFamily',
       'CEN_tr_pctSalesFamily', 'CEN_tr_pctServiceFamily',
       'CEN_tr_pctManagementFamily', 'CEN_tr_pctProductionGovt',
       'CEN_tr_pctConstructionGovt', 'CEN_tr_pctSalesGovt',
       'CEN_tr_pctServiceGovt', 'CEN_tr_pctManagementGovt',
       'CEN_tr_pctProductionNonProfit'],
      dtype='object') gives the best performance 

Now processing iteration 20 

Conclusion: cluster based on variable Index(['JointInd', 'MarkettoArea_C1',
       'Housing.Units.Percent.Built.2000.to.2004_C3',
       'Housing.Units.Percent.5.Units_C3', 'Number.of.Lifestyles.All_C3',
       'CEN_tr_pctGE200KAge65p

Conclusion: cluster based on variable Index(['JointInd', 'MarkettoArea_C1',
       'Housing.Units.Percent.Built.2000.to.2004_C3',
       'Housing.Units.Percent.5.Units_C3', 'Number.of.Lifestyles.All_C3',
       'CEN_tr_pctGE200KAge65plus', 'CEN_tr_pctLT50KAge65plus',
       'CEN_tr_pctLT25KAge65plus', 'CEN_tr_pctLT10KAge65plus',
       'CEN_tr_pctProductionFamily', 'CEN_tr_pctConstructionFamily',
       'CEN_tr_pctSalesFamily', 'CEN_tr_pctServiceFamily',
       'CEN_tr_pctManagementFamily', 'CEN_tr_pctProductionGovt',
       'CEN_tr_pctConstructionGovt', 'CEN_tr_pctSalesGovt',
       'CEN_tr_pctServiceGovt', 'CEN_tr_pctManagementGovt',
       'CEN_tr_pctProductionNonProfit', 'CEN_tr_pctConstructionNonProfit',
       'CEN_tr_pctSalesNonProfit', 'CEN_tr_pctServiceNonProfit',
       'CEN_tr_pctManagementNonProfit', 'CEN_tr_pctProductionSelfEmp',
       'CEN_tr_pctConstructionSelfEmp', 'CEN_tr_pctSalesSelfEmp',
       'CEN_tr_pctServiceSelfEmp', 'CEN_tr_pctManagementSelfEmp',
       'CEN_t

Conclusion: cluster based on variable Index(['JointInd', 'MarkettoArea_C1',
       'Housing.Units.Percent.Built.2000.to.2004_C3',
       'Housing.Units.Percent.5.Units_C3', 'Number.of.Lifestyles.All_C3',
       'CEN_tr_pctGE200KAge65plus', 'CEN_tr_pctLT50KAge65plus',
       'CEN_tr_pctLT25KAge65plus', 'CEN_tr_pctLT10KAge65plus',
       'CEN_tr_pctProductionFamily', 'CEN_tr_pctConstructionFamily',
       'CEN_tr_pctSalesFamily', 'CEN_tr_pctServiceFamily',
       'CEN_tr_pctManagementFamily', 'CEN_tr_pctProductionGovt',
       'CEN_tr_pctConstructionGovt', 'CEN_tr_pctSalesGovt',
       'CEN_tr_pctServiceGovt', 'CEN_tr_pctManagementGovt',
       'CEN_tr_pctProductionNonProfit', 'CEN_tr_pctConstructionNonProfit',
       'CEN_tr_pctSalesNonProfit', 'CEN_tr_pctServiceNonProfit',
       'CEN_tr_pctManagementNonProfit', 'CEN_tr_pctProductionSelfEmp',
       'CEN_tr_pctConstructionSelfEmp', 'CEN_tr_pctSalesSelfEmp',
       'CEN_tr_pctServiceSelfEmp', 'CEN_tr_pctManagementSelfEmp',
       'CEN_t

Conclusion: cluster based on variable Index(['JointInd', 'MarkettoArea_C1',
       'Housing.Units.Percent.Built.2000.to.2004_C3',
       'Housing.Units.Percent.5.Units_C3', 'Number.of.Lifestyles.All_C3',
       'CEN_tr_pctGE200KAge65plus', 'CEN_tr_pctLT50KAge65plus',
       'CEN_tr_pctLT25KAge65plus', 'CEN_tr_pctLT10KAge65plus',
       'CEN_tr_pctProductionFamily', 'CEN_tr_pctConstructionFamily',
       'CEN_tr_pctSalesFamily', 'CEN_tr_pctServiceFamily',
       'CEN_tr_pctManagementFamily', 'CEN_tr_pctProductionGovt',
       'CEN_tr_pctConstructionGovt', 'CEN_tr_pctSalesGovt',
       'CEN_tr_pctServiceGovt', 'CEN_tr_pctManagementGovt',
       'CEN_tr_pctProductionNonProfit', 'CEN_tr_pctConstructionNonProfit',
       'CEN_tr_pctSalesNonProfit', 'CEN_tr_pctServiceNonProfit',
       'CEN_tr_pctManagementNonProfit', 'CEN_tr_pctProductionSelfEmp',
       'CEN_tr_pctConstructionSelfEmp', 'CEN_tr_pctSalesSelfEmp',
       'CEN_tr_pctServiceSelfEmp', 'CEN_tr_pctManagementSelfEmp',
       'CEN_t

Conclusion: cluster based on variable Index(['JointInd', 'MarkettoArea_C1',
       'Housing.Units.Percent.Built.2000.to.2004_C3',
       'Housing.Units.Percent.5.Units_C3', 'Number.of.Lifestyles.All_C3',
       'CEN_tr_pctGE200KAge65plus', 'CEN_tr_pctLT50KAge65plus',
       'CEN_tr_pctLT25KAge65plus', 'CEN_tr_pctLT10KAge65plus',
       'CEN_tr_pctProductionFamily', 'CEN_tr_pctConstructionFamily',
       'CEN_tr_pctSalesFamily', 'CEN_tr_pctServiceFamily',
       'CEN_tr_pctManagementFamily', 'CEN_tr_pctProductionGovt',
       'CEN_tr_pctConstructionGovt', 'CEN_tr_pctSalesGovt',
       'CEN_tr_pctServiceGovt', 'CEN_tr_pctManagementGovt',
       'CEN_tr_pctProductionNonProfit', 'CEN_tr_pctConstructionNonProfit',
       'CEN_tr_pctSalesNonProfit', 'CEN_tr_pctServiceNonProfit',
       'CEN_tr_pctManagementNonProfit', 'CEN_tr_pctProductionSelfEmp',
       'CEN_tr_pctConstructionSelfEmp', 'CEN_tr_pctSalesSelfEmp',
       'CEN_tr_pctServiceSelfEmp', 'CEN_tr_pctManagementSelfEmp',
       'CEN_t

In [14]:
res = ['JointInd', 'MarkettoArea_C1',
       'Housing.Units.Percent.Built.2000.to.2004_C3',
       'Housing.Units.Percent.5.Units_C3', 'Number.of.Lifestyles.All_C3',
       'CEN_tr_pctGE200KAge65plus', 'CEN_tr_pctLT50KAge65plus',
       'CEN_tr_pctLT25KAge65plus', 'CEN_tr_pctLT10KAge65plus',
       'CEN_tr_pctProductionFamily', 'CEN_tr_pctConstructionFamily',
       'CEN_tr_pctSalesFamily', 'CEN_tr_pctServiceFamily',
       'CEN_tr_pctManagementFamily', 'CEN_tr_pctProductionGovt',
       'CEN_tr_pctConstructionGovt', 'CEN_tr_pctSalesGovt',
       'CEN_tr_pctServiceGovt', 'CEN_tr_pctManagementGovt',
       'CEN_tr_pctProductionNonProfit', 'CEN_tr_pctConstructionNonProfit',
       'CEN_tr_pctSalesNonProfit', 'CEN_tr_pctServiceNonProfit',
       'CEN_tr_pctManagementNonProfit', 'CEN_tr_pctProductionSelfEmp',
       'CEN_tr_pctConstructionSelfEmp', 'CEN_tr_pctSalesSelfEmp',
       'CEN_tr_pctServiceSelfEmp', 'CEN_tr_pctManagementSelfEmp',
       'CEN_tr_pctProductionPrivate', 'CEN_tr_pctConstructionPrivate',
       'CEN_tr_pctSalesPrivate', 'CEN_tr_pctServicePrivate',
       'CEN_tr_pctManagementPrivate', 'CEN_tr_pctAdministrationProd',
       'CEN_tr_pctOtherProd', 'CEN_tr_pctArtsProd', 'CEN_tr_pctEducationProd',
       'CEN_tr_pctProfessionalProd', 'CEN_tr_pctFinanceProd',
       'CEN_tr_pctInformationProd', 'CEN_tr_pctTransportationProd',
       'CEN_tr_pctRetailProd', 'CEN_tr_pctWholesaleProd',
       'CEN_tr_pctConstructionProd', 'CEN_tr_pctAgricultureProd',
       'CEN_tr_pctAdministrationCon', 'CEN_tr_pctPopBelowPovertyLevel',
       'CEN_tr_pctDoctorateDegree', 'CEN_tr_pctProfessionalDegree']
res2 = {}
df_dic = pd.read_excel("/data/capstone_data/DataDictionary_allPHB_allvendors_cleaned.xlsx")
for column in res:
    res2[column] = df_dic.loc[df_dic['Variable'] == column, 'Description'].item()
    print(df_dic.loc[df_dic['Variable'] == column, 'Description'].item())
selected_feature = pd.DataFrame.from_dict(res2, orient='index')
selected_feature.reset_index(level=0, inplace=True)
selected_feature.columns = ['Variable', 'Description']

Indicator of a joint contract
Self explanatory
Self explanatory
Self explanatory
Self explanatory
Percentage of Households With In The Past 12 Months Greater Than $200,000 among Householders  65 years and over
Percentage of Households With In The Past 12 Months Less than $50,000 among Householders  65 years and over
Percentage of Households With In The Past 12 Months Less than $25,000 among Householders  65 years and over
Percentage of Households With In The Past 12 Months Less than $10,000 among Householders  65 years and over
Percentage of People work as Self-employed in own not incorporated business workers and unpaid family workers with Production, transportation, and material moving occupations
Percentage of People work as Self-employed in own not incorporated business workers and unpaid family workers with Natural resources, construction, and maintenance occupations
Percentage of People work as Self-employed in own not incorporated business workers and unpaid family workers with 

In [15]:
pd.set_option('display.max_colwidth', -1) 
selected_feature

,Variable,Description
0,JointInd,Indicator of a joint contract
1,MarkettoArea_C1,Self explanatory
2,Housing.Units.Percent.Built.2000.to.2004_C3,Self explanatory
3,Housing.Units.Percent.5.Units_C3,Self explanatory
4,Number.of.Lifestyles.All_C3,Self explanatory
5,CEN_tr_pctGE200KAge65plus,"Percentage of Households With In The Past 12 Months Greater Than $200,000 among Householders 65 years and over"
6,CEN_tr_pctLT50KAge65plus,"Percentage of Households With In The Past 12 Months Less than $50,000 among Householders 65 years and over"
7,CEN_tr_pctLT25KAge65plus,"Percentage of Households With In The Past 12 Months Less than $25,000 among Householders 65 years and over"
8,CEN_tr_pctLT10KAge65plus,"Percentage of Households With In The Past 12 Months Less than $10,000 among Householders 65 years and over"
9,CEN_tr_pctProductionFamily,"Percentage of People work as Self-employed in own not incorporated business workers and unpaid family workers with Production, transportation, and material moving occupations"


In [16]:
selected_feature.to_csv('selected_feature_Kmeans_inertia.csv')